# Graphs

그래프는 정말 어려운 주제이고 많은 내용이 연관이 되어있습니다. 기본적인 내용만 살펴보면요.

- 그래프를 구성하는 방법에서부터 *adjacency list*, *adjacency matrix* 를 둘 수가 있구요. 
- 그래프 데이터베이스를 구성할 때에는 더 어렵습니다. *(V, E, V)* 의 triple notation 으로 구성하던지, *edge*, *node*의 형태로 따로 구성할 수가 있습니다.
- 그래프의 방향성이 있냐 없냐에 따라서 *direct graph*, *undirect graph* 를 둘 수가 있구요. 
- 연결됨의 방식에 따라 *strongly connected components*, *weakly connected components*를 찾아볼 수 있구요.
- 방향성이 없으며 cycle이 없는 애들을 *tree*, *forest* 라고 부르기도 하죠. 이런 애들을 찾는 재밌는 알고리즘 중에 하나가 *minimum spanning tree* 입니다.
- 방향성이 있는 애들의 줄을 세우는 것을 *topological sort* 라고 합니다.
- 그래프에서 node, edge를 읽는 방법이 여러가지가 있는데요. 물론 효율적으로요. 가장 간단하게는 *BFS*, *DFS*가 있구요. *shortest path* 찾는것도 재밌고, *maximum flow* 도 재밌습니다.

## 그래프 표현 방식

이번장에 있는 문제들은 **adjancency list** 방식을 사용합니다:
- `class GraphVertex`로 모든것을 해결합니다. 그리고, `List[GraphVertex]`로 그래프를 표현하죠.
- 리스트 문제들이 `class List`로 표현되는 것과 같은 이치죠.

```python
class GraphVertex:
    def __init__(self) -> None:
        self.label = label
        self.edges: List['GraphVertex'] = []
```

## BFS vs DFS

Graph search 에 있어서 가장 간단한 알고리즘들입니다. 18장과 leetcode에 가장 많이 있는 문제들이구요.

### How they work?

Depth를 따라서 먼저 찾아보느냐, Breadth를 우선시해서 찾아보느냐의 차이일 뿐이죠. 자세한 구현은 아래 예제들이나 인터넷을 참고하세요.
- DFS는 먼저 깊이를 가기에 stack을 사용합니다. 명시적으로 stack을 사용해서 node를 탐색하던, recursion을 사용하면 됩니다.
- BFS는 먼저 가까운 모든 노드들을 가기에 queue를 사용하여 iterative한 구현을 하면 됩니다.

### Complexity

복잡도는 항상 같습니다. 외워두세요.
- Time Complexity: O(V + E)
- Space Complexity: O(V)

### Application

- DFS: **Looking for cycles**, **Looking for connected components**
- BFS: **Optimization**, **Finding path quickly**

## 18.0 Can Team A beat Team B?

- 문제: 스포츠 경기에서 여러팀들의 경기 결과가 있다. 예들들어, A beats B, B beats C, C beats D... 이런식이다. 임의의 팀 A, B에 대해서 A > C > ... > B 의 형태인 transitively relative besting 을 어떻게 찾을까? 결과를 T/F로 리턴하자.
- 타입: **BFS/DFS (basic search)**
- 복잡도: O(V+E) for Time, O(V) for Space

In [1]:
import collections

MatchResult = collections.namedtuple('MatchResult', ('winning_team', 'losing_team'))


def can_tam_a_beat_team_b(matches, team_a, team_b):
    def build_graph():
        g = collections.defaultdict(set)
        for match in matches:
            g[match.winning_team].add(match.losing_team)
        return g
    
    def dfs(g, curr, dest, visited=set()):
        if curr == dest:
            return True
        elif curr in visited or curr not in graph:
            return False
        visited.add(curr)
        return any(dfs(g, team, dest, visited) for team in graph[curr])
    
    return dfs(build_graph(), team_a, team_b)

## 18.4 Deadlock Detection

- 문제: deadlock은 여러 lock들간에 cycle이 생겨서 발생할 수 있다. 주어진 그래프 g 에서 사이클이 있는지 확인하는 프로그램을 짜라.
- 타입: **DFS with marking (looking for cycles)**
- 힌트: 일반적인 DFS로는 안된다. Marking 을 3가지 타입으로 해야한다 (처음 방문, 방문중, 방문 끝)
    - WHITE: 처음방문
    - GRAY: 방문중
    - BLACK: 방문끝
- 복잡도: O(V+E) for Time, O(V) for Space

In [2]:
from typing import List


class GraphVertex:
    WHITE, GRAY, BLACK = range(3)
    
    def __init__(self) -> None:
        self.color = GraphVertex.WHITE
        self.edges: List['GraphVertex'] = []
            
            
def is_deadlocked(graph: List[GraphVertex]) -> bool:
    def has_cycle(curr):
        if curr == GraphVertex.GRAY:
            return True
        
        curr.color = GraphVertex.GRAY
        if any(nxt.color != GraphVertex.BLACK and has_cycle(nxt)
               for nxt in cur.edges):
            return True
        curr.color = GraphVertex.BLACK
        return False
    
    return any(vertex.color == GraphVertex.WHITE and has_cycle(vertex)
               for vertex in graph)

## 18.5 Clone a Graph

- 문제: Digraph를 어떻게 clone하냐?
- 타입: **BFS/DFS (navigate graph with map)**
- 복잡도: O(V+E) for Time, O(V) for Space

In [3]:
from typing import List
import collections


class GraphVertex:
    def __init__(self) -> None:
        self.label = label
        self.edges: List['GraphVertex'] = []

            
def clone_graph(graph: GraphVertex) -> GraphVertex:
    if graph is None:
        return None
    
    q = collections.deque([graph])
    vertex_map = {graph: GraphVertex(graph.label)}
    while q:
        v = q.popleft()
        for e in v.edges:
            if e not in vertex_map:
                vertex_map[e] = GraphVertex(e.label)
                q.append(e)
            vertex_map[v].edges.append(vertex_map[e])
    return vertex_map[graph]

## 18.6 Making Wired Connections

- 문제: PCB 빵판이 있고, 각핀을 두개의 색으로만 칠할 수 있다. 핀의 연결을 다른 색으로만 가능하다고 할때, 이 그래프가 제대로 구성된것인지 체크하라.
- 타입: **BFS, Bi-partite graph**
- 복잡도: O(V+E) for Time, O(V) for Space

In [4]:
from typing import List
import collections


class GraphVertex:
    def __init__(self) -> None:
        self.edges: List['GraphVertex'] = []


def is_placement_feasible(graph: List[GraphVertex]) -> bool:
    placements = collections.defaultdict(set)
    
    def bfs(v: GraphVertex):
        """한 트리에 대해서 WHITE, BLACK을 다 assign 해줌"""
        if v in placements['WHITE'] or v in placements['BLACK']:
            return True
        
        q = collections.deque([v])
        placements['WHITE'].add(v)
        
        while q:
            curr = q.popleft()
            curr_color = 'WHITE' if curr in placements['WHITE'] else 'BLACK'
            next_color = 'WHITE' if curr_color == 'WHITE' else 'BLACK'
            for n in curr.edges:
                if n in placements[curr_color]:
                    return False
                elif n not in placements[next_color]:
                    placements[next_color].add(n)
                    q.append(n)
        return True
    return all(bfs(v) for v in graph)

## 18.7 Word Ladder

Leetcode [127. Word Ladder](https://leetcode.com/problems/word-ladder/), [126. Word Ladder II](https://leetcode.com/problems/word-ladder-ii/) 에 있는 문제들입니다. 첫문제는 가능한 경우 가장 짧은 변환 숫자를 반환하는 것이고, 두번째 문제는 변환 가능한 모든 리스트를 반환하는 문제입니다.

- 문제: 사전이 주어져있고, 시작단어, 끝단어가 있다. 단어에서 매 턴마다 한글자를 바꿀수가 있다. 사전에 그 단어가 있으면 그 턴에서 다음 단어로 넘어간다. 그랬을때에 시작 단어에서 끝 단어까지 갈 수 있는가? 가능하면 가장 짧게 몇번만에 갈 수 있는가?
- 타입: **BFS (finding path quickly)**
- 풀이방법:
    - 내방법: 아래의 코드는 사전을 우선적으로 preprocessing 함
    - 책방법: 매턴마다 단어에서 새 단어를 만들어냄. 그럼, 매턴마다 (length of word) * (26 characters) 연산을 해야함.
- 복잡도:
    - V = number of words in dictionary
    - E = V\*V in the worst case
    - 내방법: O($V$) for space, O($V+E$) = O($V^2$) for time
    - 책방법: O($V$) for space, O($V+E$) = O($V^2$) for time

In [5]:
from typing import List
import collections


def ladder_length(beginWord: str, endWord: str, wordList: List[str]) -> int:
    if not wordList:
        return 0

    word_len = len(wordList[0])
    word_dict = collections.defaultdict(list)
    for word in wordList:
        for i in range(word_len):
            word_dict['_'.join([word[:i], word[i+1:]])].append(word)
    visited = {beginWord}
    queue = collections.deque([(beginWord, 1)])

    while queue:
        word, count = queue.popleft()
        if word == endWord:
            return count
        for i in range(word_len):
            next_cand = '_'.join([word[:i], word[i+1:]])
            if next_cand in word_dict:
                next_words = [word for word in word_dict[next_cand] if word not in visited]
                if next_words:
                    visited.update(next_words)
                    queue.extend([(word, count+1) for word in next_words])

    return 0

In [6]:
assert ladder_length('hit', 'cog', ["hot","dot","dog","lot","log","cog"]) == 5
assert ladder_length('hit', 'cog', ["hot","dot","dog","lot","log"]) == 0

## 18.8 Team Photo Day 2

- 문제: 18.0의 경기에 이기는 순서를 정하는 문제와 비슷하다. 단, "가장 긴" 리스트를 찾는것이다. 그러려면 BFS, DFS를 사용해서 모든 path를 탐색한 후에 가장 긴 path를 반환하면 된다.
- 타입: **Topological sort (finding paths)**
- 복잡도: O(V+E) for Time, O(V) for Space

In [7]:
from typing import List
import collections


class GraphVertex:
    def __init__(self) -> None:
        self.max_depth = 0
        self.edges: List['GraphVertex'] = []


def find_largest_number_of_teams(graph: List[GraphVertex]) -> int:
    def dfs(curr: GraphVertex) -> int:
        curr.max_depth = max((curr.max_depth if curr.max_depth > 0 else max(dfs(losers)) + 1
                              for losers in curr.edges), default=1)
        return curr.max_depth
    
    return max(dfs(team) for team in graph)

## TODO: Advanced Graph Algorithms

- adjancency list, matrix
- Bi-partite graphs
- Topological sort
- Strongly/weakly connected components
- Shortest path
- Minimum spanning tree
- Matching
- Maximum flow